# 6D-MAN Basic Usage Example

This notebook demonstrates the basic usage of the 6D-MAN package for optical metro-urban network planning.

In [ ]:
import sys
import os
# Navigate relative to the current working directory
sys.path.append(os.path.abspath('../src'))
import numpy as np
import pandas as pd
from sixdman.core.network import Network
from sixdman.core.band import Band, OpticalParameters
from sixdman.core.planning import PlanningTool
import json
from sixdman.utils.paths import get_project_root

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Use it to get paths
project_root = get_project_root()
results_dir = project_root / "results"
results_dir.mkdir(parents=True, exist_ok=True)

## 1. Create Network Instance

In [3]:
# Initialize network
network = Network(topology_name = 'MAN157')

# Load topology from .mat file
network.load_topology(filepath = '../data/MAN157Nodes.mat', matrixName ='MAN157Nodes')

# Set hierarchical levels
hl_dict = network.define_hierarchy(
    HL1_standalone = [1, 5],
    HL2_standalone = [0, 2, 3, 4],
    HL3_standalone = list(range(6, 39)),
    HL4_standalone = list(range(39, 157))
)

In [4]:
HL4_Standalone = hl_dict['HL4']['standalone']
HL4_colocated = hl_dict['HL4']['colocated']
HL4_all = np.concatenate((HL4_Standalone, HL4_colocated))

In [5]:
_, subnetMatrix_HL4 = network.compute_hierarchy_subgraph(hierarchy_level = 4, minimum_hierarchy_level = 4)
HL4_connected_nodes = network.get_neighbor_nodes(HL4_Standalone)

Standalone HL4s

In [6]:
file_name = f"./../results/{network.topology_name}_HL4_K_path_attributes.csv"

if os.path.exists(file_name):
    K_path_attributes_df = pd.read_csv(file_name)
    K_path_attributes_df['links'] = K_path_attributes_df['links'].map(json.loads)
    K_path_attributes_df['nodes'] = K_path_attributes_df['nodes'].map(json.loads)
else:
    
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes = []

    # iterate through each standalone HL4 node
    for src in HL4_Standalone:
        for dest in HL4_connected_nodes:
            K_path_attributes = network.compute_k_shortest_paths(subnetMatrix_HL4, K_path_attributes, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_df = pd.DataFrame(K_path_attributes)

    # save dataframe to csv file
    K_path_attributes_df.to_csv(file_name, index = False)

Colocated HL4s

In [7]:
file_name = f"./../results/{network.topology_name}_HL4_K_path_attributes_colocated.csv"

if os.path.exists(file_name):
    K_path_attributes_colocated_df = pd.read_csv(file_name)
    K_path_attributes_colocated_df['links'] = K_path_attributes_colocated_df['links'].map(json.loads)
    K_path_attributes_colocated_df['nodes'] = K_path_attributes_colocated_df['nodes'].map(json.loads)
else:
    
    k_paths = 20

    # define a list to store path attributes
    K_path_attributes_colocated = []

    # iterate through each standalone HL4 node
    for src in HL4_colocated:
        for dest in HL4_connected_nodes:
            if src != dest:
                K_path_attributes_colocated = network.compute_k_shortest_paths(subnetMatrix_HL4, K_path_attributes_colocated, source = src, target = dest, k = k_paths)

    # Convert K_path_attributes list to dataframe
    K_path_attributes_colocated_df = pd.DataFrame(K_path_attributes_colocated)

    # save dataframe to csv file
    K_path_attributes_colocated_df.to_csv(file_name, index = False)

In [8]:
# sort dataframes based on num_hops and distance (in order)
K_path_attributes_df_sorted = K_path_attributes_df.groupby(['src_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))
K_path_attributes_colocated_df_sorted = K_path_attributes_colocated_df.groupby(['src_node', 'dest_node'], group_keys = False).apply(lambda x: x.sort_values(['num_hops', 'distance']))

In [9]:
K_path_attributes_df_sorted

,src_node,dest_node,nodes,links,distance,num_hops
0,39,4,"[39, 40, 42, 4]","[118, 120, 27]",30.0,3
9,39,5,"[39, 41, 44, 45, 5]","[119, 121, 123, 31]",37.0,4
10,39,5,"[39, 41, 44, 43, 5]","[119, 121, 122, 30]",38.0,4
11,39,5,"[39, 40, 42, 4, 53, 52, 54, 5]","[118, 120, 27, 28, 131, 132, 33]",72.2,7
1,39,4,"[39, 41, 44, 45, 5, 54, 52, 53, 4]","[119, 121, 123, 31, 33, 132, 131, 28]",79.2,8
...,...,...,...,...,...,...
1731,156,3,"[156, 154, 3]","[218, 23]",30.0,2
1730,156,0,"[156, 155, 0]","[219, 6]",32.0,2
1732,156,14,"[156, 154, 3, 100, 95, 14]","[218, 23, 22, 172, 64]",66.0,5
1733,156,14,"[156, 154, 3, 99, 101, 102, 100, 95, 14]","[218, 23, 21, 175, 177, 176, 172, 64]",91.0,8


In [10]:
pairs_disjoint = network.land_pair_finder(HL4_Standalone, K_path_attributes_df_sorted, num_pairs = 1)

## 2. Define Transmission Bands

In [11]:
# Define C-band parameters
c_band_params = OpticalParameters()

# Create C-band instance
c_band = Band(
    name='C',
    start_freq = 190.65, # THz
    end_freq = 196.675, # THz
    opt_params = c_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
    )

# Define L-band parameters
l_band_params = OpticalParameters()

# Create L-band instance
l_band = Band(
    name='L',
    start_freq = 184.525, # THz
    end_freq = 190.565, # THz
    opt_params = l_band_params,
    network_instance = network,
    channel_spacing = 0.05 # THz
)

In [12]:
# define C-band and L-band frequency slots
spectrum_C = c_band.calc_spectrum()
spectrum_L = l_band.calc_spectrum()

# concatenate C-band and KL-band to a sigle frequency spectrum
spectrum = np.concatenate((spectrum_C, spectrum_L))

# define total number of frequency slots
num_fslots = len(spectrum)

f_c_axis = spectrum * 1e12  # Convert to Hz
Pch_dBm = np.arange(-6, -0.9, 0.1)  # Channel power in dBm
num_Ch_mat = np.arange(1, len(spectrum) - 1)  # Channel indices

In [13]:
GSNR_opt_link, _, _, _ = c_band.process_link_gsnr(f_c_axis = f_c_axis, 
                                                  Pch_dBm = Pch_dBm, 
                                                  num_Ch_mat = num_Ch_mat,
                                                  spectrum_C = spectrum_C,
                                                  Nspan_array = np.ones(network.all_links.shape[0], dtype=int),
                                                  hierarchy_level = 4, 
                                                  minimum_hierarchy_level = 4,
                                                  result_directory = results_dir)

Loading precomputed link GSNR analysis


In [14]:
np.load(r'D:\University\Master\Project\Codes\OpenSourcePlanner\results\Man157\NoBypass\MAN157_process_GSNR_HL4.npz')['GSNR_opt_link']

array([[38.13643937, 38.04594415, 38.0030148 , ..., 38.53300727,
        38.58509191, 38.68786173],
       [38.8119674 , 38.71301679, 38.66531612, ..., 38.99665938,
        39.04852351, 39.15137073],
       [38.47457508, 38.37989295, 38.33459198, ..., 38.76591938,
        38.81791316, 38.92075208],
       ...,
       [38.31848784, 38.22359711, 38.17832951, ..., 38.64959866,
        38.70282555, 38.80816437],
       [37.48371905, 37.39715576, 37.35679464, ..., 38.06205343,
        38.11659788, 38.22417695],
       [38.13643937, 38.04594415, 38.0030148 , ..., 38.53300727,
        38.58509191, 38.68786173]])

## 3. Create Planning Tool and Optimize Network

In [15]:
# Initialize planning tool
planner = PlanningTool(
    network_instance = network,
    bands = [c_band, l_band], 
    period_time = 10)


#### *Simulating aggregated traffic at HLs*

In [16]:
planner.initialize_planner(num_fslots = num_fslots,
                           hierarchy_level = 4, 
                           minimum_hierarchy_level = 4)

In [17]:
# generate port capacity for HL4 nodes uisng Monte Carlo simulation
planner.generate_initial_traffic_profile(num_nodes = len(HL4_all),
                                monteCarlo_steps = 100,
                                min_rate = 20, # Gbps
                                max_rate = 200, # Gbps
                                seed = 20, result_directory = results_dir)

# Traffic growth simulation over 10 years
planner.simulate_traffic_annual(lowest_hierarchy_dict = hl_dict['HL4'],
                                CAGR = 0.4, 
                                result_directory = results_dir)

Loading precomputed HL_capacity_final ...
Loading precomputed Traffic Matrix ...


In [18]:
planner.run_planner(hierarchy_level = 4,
                    prev_hierarchy_level = 3,
                    pairs_disjoint = pairs_disjoint,
                    kpair_standalone = 1,
                    kpair_colocated = 1,
                    candidate_paths_standalone_df = K_path_attributes_df,
                    candidate_paths_colocated_df = K_path_attributes_colocated_df,
                    GSNR_opt_link = GSNR_opt_link,
                    minimum_level = 4, 
                    node_cap_update_idx = 2,
                    result_directory = results_dir)

Processing Year:  1
Processing Year:  2
Processing Year:  3
Processing Year:  4
Processing Year:  5
Processing Year:  6
Processing Year:  7
Processing Year:  8
Processing Year:  9
Processing Year:  10
